In [1]:
#install packages

!pip install gensim
!pip install wordcloud
!pip install spacy
!pip install nltk

     |████████████████████████████████| 24.2 MB 21.5 MB/s eta 0:00:01
     |████████████████████████████████| 116 kB 109.0 MB/s eta 0:00:01
  Created wheel for smart-open: filename=smart_open-2.1.0-py3-none-any.whl size=110317 sha256=097a06e0f2987228c97ce99df89437e7a3b8899c0255a903c37e16ba63e581b5
  Stored in directory: /home/ec2-user/.cache/pip/wheels/a4/9b/d5/85705a7ab783cd6f7bd718f01d3b1396272f30044e3c36401a
Successfully built smart-open
You should consider upgrading via the '/home/ec2-user/anaconda3/envs/python3/bin/python -m pip install --upgrade pip' command.
     |████████████████████████████████| 364 kB 20.2 MB/s eta 0:00:01
You should consider upgrading via the '/home/ec2-user/anaconda3/envs/python3/bin/python -m pip install --upgrade pip' command.
     |████████████████████████████████| 9.9 MB 19.2 MB/s eta 0:00:01
     |████████████████████████████████| 185 kB 95.3 MB/s eta 0:00:01
     |████████████████████████████████| 119 kB 106.0 MB/s eta 0:00:01
     |██████████████████

  Created wheel for wasabi: filename=wasabi-0.7.1-py3-none-any.whl size=20834 sha256=0bd6beeb87a89dc86f7313e9507018e7ba5fa33a1a4698424a6f2da732097b9c
  Stored in directory: /home/ec2-user/.cache/pip/wheels/81/48/90/cf81833b3dfce6eaf7eab4bd5fdc0e75dbca4418b263f444b8
Successfully built wasabi
You should consider upgrading via the '/home/ec2-user/anaconda3/envs/python3/bin/python -m pip install --upgrade pip' command.
You should consider upgrading via the '/home/ec2-user/anaconda3/envs/python3/bin/python -m pip install --upgrade pip' command.


In [2]:
#Load pakcages

import pandas as pd
import numpy as np

# Import the json module
import json
from datetime import datetime

# Import WHATSAPP data

In [4]:
#Load data
df=pd.read_csv('data/raw_thread_chat_20200725.txt',header=None,error_bad_lines=False,encoding='utf8')

#Remove first row
df= df.drop(0)

df.head()

b'Skipping line 6: expected 2 fields, saw 3\nSkipping line 13: expected 2 fields, saw 3\nSkipping line 18: expected 2 fields, saw 3\nSkipping line 20: expected 2 fields, saw 3\nSkipping line 29: expected 2 fields, saw 3\nSkipping line 33: expected 2 fields, saw 3\nSkipping line 46: expected 2 fields, saw 3\nSkipping line 71: expected 2 fields, saw 3\nSkipping line 74: expected 2 fields, saw 3\nSkipping line 75: expected 2 fields, saw 4\nSkipping line 84: expected 2 fields, saw 3\nSkipping line 91: expected 2 fields, saw 4\nSkipping line 92: expected 2 fields, saw 3\nSkipping line 93: expected 2 fields, saw 3\nSkipping line 94: expected 2 fields, saw 3\nSkipping line 96: expected 2 fields, saw 4\nSkipping line 99: expected 2 fields, saw 3\nSkipping line 102: expected 2 fields, saw 3\nSkipping line 126: expected 2 fields, saw 4\nSkipping line 133: expected 2 fields, saw 6\nSkipping line 135: expected 2 fields, saw 5\nSkipping line 143: expected 2 fields, saw 6\nSkipping line 170: expecte

,0,1
1,02/08/2018,16:56 - Sam;: Free transfer
2,02/08/2018,16:57 - Paul Dickinson: Oh so just load him u...
3,02/08/2018,16:57 - Rob;: 70K a week for Eriksen would ha...
4,02/08/2018,16:58 - / James Ball: <Media omitted>
5,02/08/2018,17:07 - Paul Dickinson: How big is the wall g...


In [5]:
#LOADING DATA FROM TXT FILE

#Load data
df=pd.read_csv('data/raw_thread_chat.txt',header=None,error_bad_lines=False,encoding='utf8')

#Remove first row
df= df.drop(0)

#format schema
df.columns=['date', 'chat']

Message = df["chat"].str.split("-", n = 1, expand = True)
df['time']=Message[0]
df['text']=Message[1]

Message1= df["text"].str.split(":", n = 1, expand = True)
df['text']=Message1[1]
df['name']=Message1[0]

df=df.drop(columns=['chat'])
df['text']=df['text'].str.lower()

df['text'] = df['text'].astype(str)
df['name'] = df['name'].astype(str)

#Adjust special messages
df['text'] = df['text'].str.replace('<media omitted>','MediaShared')
df['text'] = df['text'].str.replace('this message was deleted','DeletedMsg')

#Cleaning
df.name.dropna(inplace=True)
df = df.drop(df[df['text'] == 'None'].index)
df = df.drop(df.loc[df['name']=='nan'].index)
df = df.drop(df.loc[df['name']==' http'].index)

#Prepare name classes
names_dict = {
        ' Phill':'PhilH',
        ' David;':'DaveY',
        ' Jack G;':'JackG',
        ' Rob;':'RobG',
        ' / James Ball':'JamesB',
        ' Hamish Sinclair':'HamishS',
        ' Damian Rumble':'DamianR',
        ' Paul Dickinson':'PaulD',
        ' Sam;':'SamT',
        ' Stefan;':'StefanL',
        ' Paul;':'PaulD'
        }
    
df.name = df.name.map(names_dict)

b'Skipping line 5: expected 2 fields, saw 3\nSkipping line 19: expected 2 fields, saw 3\nSkipping line 35: expected 2 fields, saw 3\nSkipping line 47: expected 2 fields, saw 3\nSkipping line 55: expected 2 fields, saw 4\nSkipping line 59: expected 2 fields, saw 3\nSkipping line 79: expected 2 fields, saw 3\nSkipping line 82: expected 2 fields, saw 3\nSkipping line 89: expected 2 fields, saw 3\nSkipping line 103: expected 2 fields, saw 3\nSkipping line 145: expected 2 fields, saw 3\nSkipping line 148: expected 2 fields, saw 3\nSkipping line 150: expected 2 fields, saw 3\nSkipping line 154: expected 2 fields, saw 3\nSkipping line 161: expected 2 fields, saw 3\nSkipping line 172: expected 2 fields, saw 4\nSkipping line 182: expected 2 fields, saw 3\nSkipping line 184: expected 2 fields, saw 3\nSkipping line 186: expected 2 fields, saw 4\nSkipping line 188: expected 2 fields, saw 9\nSkipping line 208: expected 2 fields, saw 3\nSkipping line 217: expected 2 fields, saw 3\nSkipping line 229:

In [6]:
df.head()

,date,time,text,name
1,29/03/2018,09:00,hebiiiiib,DaveY
2,29/03/2018,09:01,hibeb,JackG
3,29/03/2018,09:01,let's just see. we need paul to make a clean ...,DaveY
4,29/03/2018,09:10,yeah - the big de-teather,DaveY
5,29/03/2018,09:11,electing trump doesn't sit with froust neutra...,DaveY


# Import FACEBOOK data

In [7]:
data_list = ['message_1','message_2','message_3']    
    
messages_dict = {}    
    
for i in data_list:  
    # Load the file 
    json_data = json.loads(open('data/{}.json'.format(i)).read())
    messages_dict[i] = json_data['messages']

df_FB_temp_1 = pd.DataFrame.from_dict(messages_dict['message_1'])
df_FB_temp_2 = pd.DataFrame.from_dict(messages_dict['message_2'])
df_FB_temp_3 = pd.DataFrame.from_dict(messages_dict['message_3'])

df_FB = pd.concat([df_FB_temp_1, df_FB_temp_2, df_FB_temp_3])

def ts_to_dttm(ts):
    #Convert timesstamp into a datetime
    return datetime.utcfromtimestamp(ts/1000.0).strftime('%d/%m/%Y %H:%M:%S')
    #%Y-%m-%d
df_FB['dttm'] = df_FB['timestamp_ms'].apply(lambda x: ts_to_dttm(x))
df_FB.head()

,sender_name,timestamp_ms,content,type,share,photos,sticker,reactions,gifs,videos,users,files,audio_files,dttm
0,Rob Goodway,1535487302182,Sam Thornton live at the Hollywood Bowel,Generic,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,28/08/2018 20:15:02
1,Rob Goodway,1535040601471,I am an ostrich,Generic,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,23/08/2018 16:10:01
2,Rob Goodway,1535040596758,My barber earlier was telling me about it. I'v...,Generic,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,23/08/2018 16:09:56
3,Rob Goodway,1535040569224,https://www.sickchirpse.com/premier-league-ref...,Share,{'link': 'https://www.sickchirpse.com/premier-...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,23/08/2018 16:09:29
4,Rob Goodway,1535040567240,Has there been much chat on the thread about r...,Generic,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,23/08/2018 16:09:27


In [8]:
#Prepapre FB data to the same schema as WhatsApp
df_FB = df_FB.loc[df_FB.type=='Generic']

df_FB['text'] = df_FB['content']
df_FB['name'] = df_FB['sender_name']

Message = df_FB["dttm"].str.split(" ", n = 1, expand = True)
df_FB['date']=Message[0]
df_FB['time']=Message[1]

df_FB = df_FB[['date','time','text','name']]

df_FB['text'] = df_FB['text'].astype(str)
df_FB['name'] = df_FB['name'].astype(str)

In [9]:
#Cleaning
df_FB.name.dropna(inplace=True)
df_FB = df_FB.drop(df_FB[df_FB['text'] == 'None'].index)
df_FB = df_FB.drop(df_FB.loc[df_FB['name']=='nan'].index)
df_FB = df_FB.drop(df_FB.loc[df_FB['name']==' http'].index)

#Prepare name classes
names_dict_FB = {
        'Phil WH':'PhilH',
        'David Yarrow':'DaveY',
        'Jack Gayer':'JackG',
        'Rob Goodway':'RobG',
        'James Ball':'JamesB',
        'Hamish William David':'HamishS',
        'Damian Jack Rumble':'DamianR',
        'Paul Dickinson':'PaulD',
        'Sam Sam':'SamT',
        'Stefan Liberadzki':'StefanL',
        'Verity Rose':'VerityR',
        'Zachary Lowther':'ZacL',
        'Holly Mumford':'HollyM',
        'Facebook User':'PhilH'
        }
    
df_FB.name = df_FB.name.map(names_dict_FB)

# Merge threads

In [11]:
df_the_thread = pd.concat([df,df_FB])

df_the_thread.to_csv('data/PREP_combo_thread_data_20200725.csv')

In [12]:
df_the_thread

,date,time,text,name
1,29/03/2018,09:00,hebiiiiib,DaveY
2,29/03/2018,09:01,hibeb,JackG
3,29/03/2018,09:01,let's just see. we need paul to make a clean ...,DaveY
4,29/03/2018,09:10,yeah - the big de-teather,DaveY
5,29/03/2018,09:11,electing trump doesn't sit with froust neutra...,DaveY
...,...,...,...,...
4096,03/03/2012,10:33:28,"dave, get here",PaulD
4097,03/03/2012,10:33:26,well i wont be finished until like mid may,PhilH
4098,03/03/2012,10:32:12,"i don't know, need to know so i can book it of...",PaulD
4099,03/03/2012,10:31:39,when?,PhilH
